In [1]:
import numpy as np
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [3]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [18]:
def prepare_dataset(src, dst, voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    # demo_icp_pcds = o3d.data.DemoICPPointClouds()
    # source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
    # target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

    source = o3d.geometry.PointCloud()
    target = o3d.geometry.PointCloud()

    source.points = o3d.utility.Vector3dVector(src)
    target.points = o3d.utility.Vector3dVector(dst)

    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [ ]:
voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [6]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [7]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
RegistrationResult with fitness=6.733193e-01, inlier_rmse=2.764157e-02, and correspondence_set size of 3205
Access transformation to get result.


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [23]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4

    source.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=30)
    )
    target.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=30)
    )

    # source = o3d.t.geometry.PointCloud.from_legacy(source_pcd)
    # target = o3d.t.geometry.PointCloud.from_legacy(target_pcd)

    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

In [9]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.020.
RegistrationResult with fitness=6.210275e-01, inlier_rmse=6.565175e-03, and correspondence_set size of 123482
Access transformation to get result.


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [10]:
import os

dataset_dir = '../data'
pr_data_dir = os.path.join(dataset_dir, 'place_recognition_data')
graph_data_dir = os.path.join(dataset_dir, 'graph_data')

In [11]:
test_name = os.listdir(pr_data_dir)[1]
test_name

'1710504598765810000'

In [12]:
test_dir = os.path.join(pr_data_dir, test_name)
try:
    transforms_ = np.loadtxt(os.path.join(test_dir, 'transforms.txt'))
except FileNotFoundError:
    pass

if transforms_.size == 0:
    pass
if transforms_.ndim == 1:
    transforms_ = transforms_[np.newaxis, :]

transforms_

array([[13.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [37.        , -0.        , -0.        , -2.72327728,  1.41153183,
        -0.40772474,  0.        ],
       [11.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [16.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [34.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ]])

In [13]:
cand_clouds = {}
cand_poses = {}

In [14]:
with np.load(os.path.join(test_dir, 'cloud.npz')) as CloudFile:
    ref_cloud_vis = CloudFile['arr_0']

ref_cloud = ref_cloud_vis[:, :3]
ref_cloud = ref_cloud[ref_cloud == ref_cloud].reshape((-1, 3))
ref_cloud

array([[ 0.5436    , -0.27659687,  0.3026525 ],
       [ 0.5235573 , -0.2753935 ,  0.2713867 ],
       [ 0.52194405, -0.27563274,  0.22517666],
       ...,
       [-5.0173335 , -1.0191256 , -0.07055677],
       [-5.010857  , -1.016701  , -0.12190759],
       [-5.004     , -1.0086188 , -0.16419376]], dtype=float32)

In [15]:
for idx in transforms_[:, 0]:
    cloud = np.load(os.path.join(graph_data_dir, str(int(idx)), 'cloud.npz'))['arr_0']
    pose_stamped = np.loadtxt(os.path.join(graph_data_dir, str(int(idx)), 'pose_stamped.txt'))
    cand_clouds[int(idx)] = cloud
    cand_poses[int(idx)] = pose_stamped[1:]

In [17]:
gt_poses = np.loadtxt(os.path.join(test_dir, 'gt_poses.txt'))
transforms = {}
pr_scores = np.loadtxt(os.path.join(test_dir, 'pr_scores.txt'))
ref_cloud = ref_cloud[ref_cloud == ref_cloud].reshape((-1, 3))

In [19]:
idx = transforms_[3, 0]
cloud = cand_clouds[idx][:, :3]
cloud = cloud[cloud == cloud].reshape((-1, 3))

In [20]:
voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(ref_cloud, cloud,
    voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [21]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
RegistrationResult with fitness=2.984499e-02, inlier_rmse=4.868987e-02, and correspondence_set size of 1084
Access transformation to get result.


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [24]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.020.
RegistrationResult with fitness=3.158197e-03, inlier_rmse=1.497600e-02, and correspondence_set size of 143
Access transformation to get result.


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


In [27]:
result_icp.fitness

0.003158196956646569

In [28]:
sample_pcd_data = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(sample_pcd_data.path)
pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/fragment.pcd
[Open3D INFO] Downloaded to /home/mavovk/open3d_data/download/PCDPointCloud/fragment.pcd


In [29]:
pcd.colors[1500] = [1, 0, 0]

In [35]:
print("Find its 200 nearest neighbors, and paint them blue.")
[k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[1500], 2)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]

Find its 200 nearest neighbors, and paint them blue.


In [36]:
idx

IntVector[1500, 1499]

In [42]:
def calculate_fitness(source, transformation, max_range=1):
    score = 0.0

    source_tmp = copy.deepcopy(source)
    source_tmp.transform(transformation)

    tree = o3d.geometry.KDTreeFlann(source_tmp)
    nr = 0

    for i, point in enumerate(source.points):
        [k, idx, _] = tree.search_knn_vector_3d(point, 2)

        print(point, source_tmp.points[idx[1]])
        break

In [43]:
calculate_fitness(source, result_icp.transformation)

[ 0.30265251  0.54360002 -0.27659687] [-0.10642514  1.19761121 -0.48561888]
